In [2]:
import igraph as ig
import networkx as nx
from operator import itemgetter
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.lines import Line2D
import matplotlib.path as mpath
import time
import collections
import community
import scipy.io
from networkx.algorithms.link_analysis import hits
import itertools
#1.Creation of Graph
def createGraph(file):
    G=nx.Graph()

    mat = scipy.io.loadmat(file) #matlab file containing an adjacency matrix for the coactivation 
    #network that includes the weights of each edge and the coordinates of each node in the brain

    content = list(mat.values()) #content[3] weights of the edges; content[4] coordinates
    #print(content[3])
    G = nx.from_numpy_matrix(content[3])
    #print(G.edges[0,1]['weight'])
    return [content, G]

def averageWeightedDegree(G):
    N = G.number_of_nodes()
    weightedDegree = np.zeros(N)
    #we start by computing the weighted degree for each node (sum of weights of the edges linked to that node        
    for u, v, w in G.edges(data = 'weight'):
        #add weight to both nodes that share the edge
        weightedDegree[u] += w
        weightedDegree[v] += w
    av_weightedDegree = sum(weightedDegree)/N
    return [weightedDegree, av_weightedDegree]


file, G = createGraph('Coactivation_matrix.mat')
weightedDegree, av_w_degree = averageWeightedDegree(G)


In [ ]:
Edges=list(G.edges)

Gi=ig.Graph(Edges, directed=False)


In [18]:
layt=Gi.layout('kk', dim=3) 
i=0
for node in file[4]:
    layt[i]=node
    i+=1
    

In [19]:
print(layt)

<Layout with 638 vertices and 3 dimensions>


In [20]:
print(layt[0])

[7.243636363636369, 37.01090909090908, 9.425454545454542]


In [21]:
print(file[4][0])

[ 7.24363636 37.01090909  9.42545455]


In [22]:
N=len(G.nodes)
Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in Edges:
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]  
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]  
    


In [23]:
part = community.best_partition(G) #returns dictionary assign a community (integer) to each node

values = [part.get(node) for node in G.nodes()] #creates a list with the community number for each node in the 
#order that appears in G.nodes()
        

In [48]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='andreiarog', api_key='4QtzshXlWo5mEnzFA7zo')

trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='nodes',
               marker=dict(symbol='circle',
                             size=weightedDegree*2,
                             color=values,
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5), opacity=1.0
                             ),
               hoverinfo='none'
               )
trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1), opacity=0.2,
               hoverinfo='none'
               )



axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         title="Network of brain (3D visualization)",
         width=1000,
         height=1000,
         showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    hovermode='closest',
    annotations=[
           dict(
           showarrow=False,
            text="",
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=dict(
            size=14
            )
            )
        ],    )

In [49]:
data=[trace1, trace2]
fig=go.Figure(data=data, layout=layout)

py.iplot(fig, file_id="brain1")

/home/andreia/anaconda3/lib/python3.6/site-packages/plotly/plotly/plotly.py:230: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



The draw time for this plot will be slow for clients without much RAM.


/home/andreia/anaconda3/lib/python3.6/site-packages/plotly/api/v1/clientresp.py:40: UserWarning:

Estimated Draw Time Slow

